In [46]:
from __future__ import print_function
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, Adadelta
from keras import backend as K

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [47]:
num_classes = 10
batch_size = 128
epochs = 24

img_rows, img_cols = 28, 28

In [48]:
from sklearn.model_selection import train_test_split
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print(x_train.shape)
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, train_size=50000, test_size=10000)
print(x_train.shape)
print(x_validation.shape)



(60000, 28, 28)
(50000, 28, 28)
(10000, 28, 28)


In [49]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    x_validation = x_validation.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    x_validation = x_validation.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [50]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_validation = x_validation.astype('float32')

x_train /= 255
x_test /= 255
x_validation /= 255
print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')
print(x_validation.shape, 'validation samples')


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_validation = keras.utils.to_categorical(y_validation, num_classes)



(50000, 28, 28, 1) train samples
(10000, 28, 28, 1) test samples
(10000, 28, 28, 1) validation samples


In [51]:
def create_conv_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     padding='same',
                     #kernel_regularizer=keras.regularizers.l1(l=0.00001),
                     input_shape=input_shape
                    ))
    #model.add(MaxPooling2D( pool_size=(2, 2) ))
    model.add(Conv2D(64, kernel_size=(3, 3),
                     activation='relu',
                     padding='same',
                     #kernel_regularizer=keras.regularizers.l1(l=0.00001)
                    ))
    model.add(MaxPooling2D( pool_size=(2, 2) ))
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(128,
                    activation='relu',
                   # kernel_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None), 
                    kernel_regularizer=keras.regularizers.l1(l=0.00001)
                   )
             )
    model.add(Dropout(0.5))
    model.add(Dense(num_classes,
                    activation='softmax',
                    #kernel_initializer=keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None), 
                    kernel_regularizer=keras.regularizers.l1(l=0.00001)
                   )
             )
    
    optimizer = Adam()
    %time model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy'])
    
    return model



In [52]:
model = create_conv_model()

CPU times: user 23.3 ms, sys: 0 ns, total: 23.3 ms
Wall time: 23.3 ms


In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
__________

In [54]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor="val_acc", mode=max, verbose=1, patience=3)

filepath="keras-fmnist-dnn-tuto-{epoch:02d}-{val_acc:.5f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=False, mode='max')

my_callbacks = [early_stopping, checkpoint]

In [ ]:
hist = model.fit(x_train, y_train, batch_size=batch_size, callbacks=my_callbacks, epochs=epochs, verbose=1, 
              validation_data=(x_validation, y_validation))

Train on 50000 samples, validate on 10000 samples
Epoch 1/24
50000/50000 [==============================] - 178s 4ms/step - loss: 0.6730 - acc: 0.8132 - val_loss: 0.4780 - val_acc: 0.8830

Epoch 00001: val_acc improved from -inf to 0.88300, saving model to keras-fmnist-dnn-tuto-01-0.88300.hdf5
Epoch 2/24
50000/50000 [==============================] - 176s 4ms/step - loss: 0.4919 - acc: 0.8767 - val_loss: 0.4395 - val_acc: 0.8928

Epoch 00002: val_acc improved from 0.88300 to 0.89280, saving model to keras-fmnist-dnn-tuto-02-0.89280.hdf5
Epoch 3/24
50000/50000 [==============================] - 174s 3ms/step - loss: 0.4480 - acc: 0.8886 - val_loss: 0.4248 - val_acc: 0.9036

Epoch 00003: val_acc improved from 0.89280 to 0.90360, saving model to keras-fmnist-dnn-tuto-03-0.90360.hdf5
Epoch 4/24
50000/50000 [==============================] - 175s 3ms/step - loss: 0.4217 - acc: 0.8997 - val_loss: 0.4080 - val_acc: 0.9048

Epoch 00004: val_acc improved from 0.90360 to 0.90480, saving model to

In [ ]:
model.save_weights("kera-fmnist-conv.model")

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')


plt.rcParams['figure.dpi'] = 100
plt.ion()
plt.axis('equal')

import numpy as np
epoch_list = list(range(1, len(hist.history['acc']) + 1))
plt.plot(epoch_list, hist.history['acc'], epoch_list, hist.history['val_acc'])
plt.legend('Training Accuracy', 'Validation Accuracy')
